"""
# PyLabRobot Tecan Fluent Backend Tutorial

This notebook demonstrates how to use the PyLabRobot Tecan Fluent backend for automated liquid handling.

## 0. Environment Setup

Before running this notebook, make sure you:
1. Have activated the virtual environment in PowerShell:
   ```powershell
   cd path\\to\\your\\project
   .\\venv\\Scripts\\activate
   ```
2. Started Jupyter Notebook from the activated environment:
   ```powershell
   cd ..  # Go back to main directory
   jupyter notebook
   ```
3. The notebook kernel should now be using the virtual environment with pylabrobot installed.

## 1. Setup

First, let's import the necessary modules and verify our environment:
"""


In [ ]:
import logging
import os
from pyfluent import FluentVisionX

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("FluentBackend")

# Verify we're in the correct directory
print(f"Current working directory: {os.getcwd()}")
print("Make sure you're in the PyFluent directory and pyfluent is installed.")

Current working directory: C:\Users\ssanch18\Silas-Tecan-pylabrobot\pylabrobot_TecanFluent_Sila\examples
If this isn't ending in 'pylabrobot_TecanFluent_Sila', use File -> Change Directory in Jupyter's menu.


## 2. Connecting to the Fluent

Initialize and connect to the Fluent. For development, we'll use simulation mode:

In [45]:
# Initialize Fluent
fluent = Fluent(
    num_channels=8,
    host="localhost",  # Use actual Fluent IP for real hardware
    port=50052,
    simulation_mode=True,  # Set to False for real hardware
    insecure=True
)

# Connect to the server
fluent.connect()
print(f"Initial state: {fluent.state}")


INFO:tecan.__Fluent:successfully connected to the server
INFO:FluentBackend:State changed to: EditMode
INFO:FluentBackend:Progress: 100.0%
ERROR:FluentBackend:Error from Fluent: Connection to Fluent Control is lost. Please try again later.
Adding the specified count to the semaphore would cause it to exceed its maximum count.
INFO:FluentBackend:Connected to Fluent successfully


Initial state: EditMode


INFO:FluentBackend:State changed to: RunModePreRunChecks
INFO:FluentBackend:State changed to: RunModePreparingRun
INFO:FluentBackend:State changed to: RunModeWaitingForSystem
INFO:FluentBackend:State changed to: RunModeBeginRun
INFO:FluentBackend:Progress: 0.0%
INFO:FluentBackend:State changed to: RunModeRunning
INFO:FluentBackend:Progress: 1.0%
INFO:FluentBackend:Progress: 2.0%
INFO:FluentBackend:Progress: 3.0%
INFO:FluentBackend:Progress: 4.0%
INFO:FluentBackend:Progress: 5.0%
INFO:FluentBackend:Progress: 6.0%
INFO:FluentBackend:Progress: 7.0%
INFO:FluentBackend:Progress: 8.0%
INFO:FluentBackend:Progress: 9.0%
INFO:FluentBackend:Progress: 10.0%
INFO:FluentBackend:Progress: 11.0%
INFO:FluentBackend:Progress: 12.0%
INFO:FluentBackend:Progress: 13.0%
INFO:FluentBackend:Progress: 14.0%
INFO:FluentBackend:Progress: 15.0%
INFO:FluentBackend:Progress: 16.0%
INFO:FluentBackend:Progress: 17.0%
INFO:FluentBackend:Progress: 18.0%
INFO:FluentBackend:Progress: 19.0%
INFO:FluentBackend:Progress: 2

## 3. Setting up the Execution Channel

We need to run a demo method to establish the execution channel:

In [46]:
import time

# Prepare and run demo method
print("Setting up execution channel...")
fluent.prepare_method("demo")
time.sleep(1)

fluent.fluent.run_method()
print("Demo method started")

# Wait for running state
start_time = time.time()
while time.time() - start_time < 30:
    state = fluent.state
    print(f"Current state: {state}")
    if state == "RunModeRunning":
        print("Execution channel established!")
        break
    time.sleep(0.5)

INFO:tecan.__Fluent:preparing method...


Setting up execution channel...


INFO:tecan.__Fluent:method ready to run
INFO:tecan.__Fluent:method running


Demo method started
Current state: RunModePreparingRun
Current state: RunModeWaitingForSystem
Current state: RunModeWaitingForSystem
Current state: RunModeWaitingForSystem
Current state: RunModeWaitingForSystem
Current state: RunModeWaitingForSystem
Current state: RunModeWaitingForSystem
Current state: RunModeBeginRun
Current state: RunModeRunning
Execution channel established!


## 4. Adding Labware

Now let's add some labware to our workspace:

In [47]:
# Add source plate
try:
    fluent.add_labware(
        "Source[001]",        # Name
        "96 Well Flat",       # Type
        "Nest61mm_Pos",       # Location
        3                     # Position
    )
    print("Successfully added source plate")
except Exception as e:
    print(f"Error adding source plate: {e}")

# Add target plate
try:
    fluent.add_labware(
        "Target[001]",        # Name
        "96 Well Flat",       # Type
        "Nest61mm_Pos",       # Location
        2                     # Position
    )
    print("Successfully added target plate")
except Exception as e:
    print(f"Error adding target plate: {e}")


INFO:FluentBackend:Adding labware Source[001] of type 96 Well Flat to Nest61mm_Pos
INFO:tecan.__Fluent:Labware added successfully
INFO:FluentBackend:Labware added successfully
INFO:FluentBackend:Adding labware Target[001] of type 96 Well Flat to Nest61mm_Pos
INFO:tecan.__Fluent:Labware added successfully
INFO:FluentBackend:Labware added successfully


Successfully added source plate
Successfully added target plate


## 5. Working with Tips

Let's try picking up some tips:

In [48]:
# Pick up tips with integer values
try:
    fluent.get_tips(
        diti_type=DiTi.FCA_200_UL,
        airgap_volume=10,     # Changed from float to int
        airgap_speed=70       # Already an int
    )
    print("Successfully picked up tips")
except Exception as e:
    print(f"Error getting tips: {e}")

INFO:FluentBackend:Getting tips of type DiTi.FCA_200_UL
INFO:tecan.__Fluent:done
INFO:FluentBackend:Tips picked up successfully


Successfully picked up tips


In [51]:
# Aspirate
fluent.aspirate(
    "Source[001]",  # Labware name
    volume=100      # Volume in µL
)

<coroutine object Fluent.aspirate at 0x000002A122673540>

In [ ]:
# Dispense
fluent.dispense(
    "Target[001]",  # Labware name
    volume=100      # Volume in µL
)

In [23]:

# Drop tips with async handling
import asyncio

try:
    print("Current state before dropping tips:", fluent.state)
    print("Attempting to drop tips into ThruDeckWaste...")

    # Create event loop and run the coroutine
    loop = asyncio.get_event_loop()
    loop.run_until_complete(fluent.drop_tips("ThruDeckWaste_Pos_1"))

    print("Successfully dropped tips in ThruDeckWaste")
    print("Current state after dropping tips:", fluent.state)
except Exception as e:
    print(f"Error dropping tips: {e}")


Current state before dropping tips: RunModeRunning
Attempting to drop tips into ThruDeckWaste...
Successfully dropped tips in ThruDeckWaste
Current state after dropping tips: RunModeRunning


C:\Users\ssanch18\AppData\Local\Temp\ipykernel_15700\1915564286.py:6: RuntimeWarning: coroutine 'Fluent.drop_tips' was never awaited
  fluent.drop_tips("ThruDeckWaste_Pos_2")  # Using your specific waste position


## 6. Using the CSV Protocol Runner

We can also run protocols from CSV files. First, let's look at our example protocol:

In [52]:
from simple_csv_runner import run_protocol

# Run the protocol with simulation mode OFF
success = run_protocol(
    csv_path="example_protocol.csv",
    simulation_mode=False,  # This will run on the actual Tecan
    host="localhost",
    port=50052
)

print(f"Protocol {'succeeded' if success else 'failed'}")

INFO:FluentCSV:Connecting to Fluent at localhost:50052...
INFO:tecan.__Fluent:successfully connected to the server
INFO:FluentBackend:State changed to: EditMode
INFO:FluentBackend:Progress: 100.0%
ERROR:FluentBackend:Error from Fluent: Connection to Fluent Control is lost. Please try again later.
Adding the specified count to the semaphore would cause it to exceed its maximum count.
INFO:FluentBackend:Connected to Fluent successfully
INFO:FluentCSV:Checking initial state...
INFO:FluentCSV:Initial state: EditMode
INFO:FluentCSV:Preparing demo method...
INFO:tecan.__Fluent:preparing method...
INFO:FluentBackend:State changed to: RunModePreRunChecks
INFO:tecan.__Fluent:method ready to run
INFO:FluentBackend:State changed to: RunModePreparingRun
INFO:FluentCSV:Running demo method...
INFO:tecan.__Fluent:method running
INFO:FluentCSV:Demo method started
INFO:FluentCSV:Waiting for execution channel...
INFO:FluentCSV:Current state: RunModePreparingRun
INFO:FluentBackend:State changed to: RunMo

Protocol succeeded


INFO:FluentBackend:Progress: 1.0%
INFO:FluentBackend:Progress: 2.0%
INFO:FluentBackend:Progress: 3.0%
INFO:FluentBackend:Progress: 4.0%
INFO:FluentBackend:Progress: 5.0%
INFO:FluentBackend:Progress: 6.0%
INFO:FluentBackend:Progress: 7.0%
INFO:FluentBackend:Progress: 8.0%
INFO:FluentBackend:Progress: 9.0%
INFO:FluentBackend:Progress: 10.0%
INFO:FluentBackend:Progress: 11.0%
INFO:FluentBackend:Progress: 12.0%
INFO:FluentBackend:Progress: 13.0%


## 7. Cleanup

Always disconnect when you're done:

In [53]:
# Fourteenth cell - code
fluent.disconnect()
print("Disconnected from Fluent")

INFO:FluentBackend:Disconnected from Fluent server


Disconnected from Fluent


# Notes

1. For real hardware:
   - Set `simulation_mode=False`
   - Use the actual IP address of your Fluent
   - Ensure the SiLA server is running

2. Common issues:
   - Make sure the demo method is loaded in Fluent
   - Check that labware positions exist
   - Verify tip types are available

3. For more examples:
   - Check the `examples/` directory
   - Look at `fluent_tests.py